In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn import preprocessing
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
p1 = pd.read_csv('data/Responses/responsesP1(Jan-Feb).csv')
p2 = pd.read_csv('data/Responses/responsesP2(Mar-Apr).csv')
p3 = pd.read_csv('data/Responses/responsesP3(May-Jun).csv')
p4 = pd.read_csv('data/Responses/responsesP4(Jul-Aug).csv')
p4 = pd.read_csv('data/Responses/responsesP4(Jul-Aug).csv')
p5 = pd.read_csv('data/Responses/responsesP5(Sep-Oct).csv')
p6 = pd.read_csv('data/Responses/responsesP6(Nov-Dec).csv')

In [ ]:
#use this for performance
q2 = pd.read_csv('data/Incidents/incidentsQ2.csv', index_col='callstart', infer_datetime_format=True, parse_dates=True)

In [ ]:
q1 = pd.read_csv('data/Incidents/incidentsQ1.csv', index_col='callstart', infer_datetime_format=True, parse_dates=True)

In [ ]:
p1_performance = (q1.groupby(q1.index.date)['response_aqi'].sum() / 3600)[-31:]

In [ ]:
p2_performance = (q2.groupby(q2.index.date)['response_aqi'].sum() / 3600)[:30]

In [ ]:
response_aqi = pd.concat([p1_performance, p2_performance], axis=0)

In [ ]:
p2_performance = (p2.groupby('date')['jobcycle'].sum() - p2.groupby('date')['hospitalturnaround'].sum()) / 3600

In [ ]:
a = pd.concat([response_aqi, p2_performance], axis=1)

In [ ]:
a.corr()

In [ ]:
p1.columns

In [ ]:
responses = pd.concat([p1, p2, p3, p4, p5, p6])

In [ ]:
capacity = pd.read_excel('data/Capacity/las_daily_ambulance_capacity_data.xlsx')

In [ ]:
capacity.head()

In [ ]:
p2.columns

In [ ]:
responses.tail()

In [ ]:
#get rid of time, just date responses
p2 = responses
p2['date'] = pd.to_datetime(p2.callstart).dt.date
#responses['date'] = pd.to_datetime(responses.callstart).dt.date

In [ ]:
#jobcycle sans hospitalturnaround / 3600
p2_performance = (p2.groupby('date')['jobcycle'].sum() - p2.groupby('date')['hospitalturnaround'].sum()) / 3600

In [ ]:
#calculate overall speed of the day
p2_to_incident = p2.groupby('date')['activation', 'mobilisation', 'running', 'tohospital'].sum()
p2_to_incident['drive_time'] = p2_to_incident.iloc[:, 0:4].sum(axis=1)
p2_distance = p2.groupby('date')['distance_to_scene'].sum()
p2_speed = p2_distance / p2_to_incident['drive_time']

In [ ]:
#calculate hospital availability
p2_hospital = p2.groupby('date')['hospitalturnaround'].sum() / 3600

In [ ]:
#concat and merge to get all the factors
p2_scm = pd.concat([p2_speed, p2_hospital, p2_performance], axis=1).rename(columns={0:'speed', 1:'performance'})
p2_scm.index = pd.to_datetime(p2_scm.index)
p2_scm = pd.merge(p2_scm, capacity.loc[:, ['Date', 'DCA aPFVH']], left_index=True, right_on='Date')

In [ ]:
#rename 'DCA aPFVH' to 'capacity'
p2_scm = p2_scm.rename(columns={'DCA aPFVH':'capacity'})
#set Date to index
p2_scm.set_index('Date', inplace=True)

In [ ]:
p2_scm.head()

In [ ]:
#standardize the dataset
scaler = preprocessing.StandardScaler()
p2_scm_normalized = scaler.fit_transform(p2_scm)
p2_scm_normalized = pd.DataFrame(p2_scm_normalized, columns=p2_scm.columns)

In [ ]:
p2_scm_normalized.head()

In [ ]:
sns.pairplot(p2_scm_normalized)

In [ ]:
model = sm.formula.ols('performance ~ speed + hospitalturnaround + capacity', data=p2_scm_normalized)
result = model.fit()
result.summary()

In [ ]:
model = sm.formula.ols('response_aqi ~ speed', data=p2_scm_normalized)
result = model.fit()
result.summary()

In [ ]:
model = sm.formula.ols('performance ~ speed + hospitalturnaround + capacity', data=p2_scm_normalized)
result = model.fit()
result.summary()

In [ ]:
result.summary()

In [ ]:
model = sm.formula.ols('performance ~ speed + hospitalturnaround + capacity', data=p2_scm_normalized)
result = model.fit()
result.params

In [ ]:
result.summary()